In [ ]:
# from src.data.datasets.nuscenes import NuScenesValidationMini, LABEL_ID2NAME
from src.data.datasets.waymo import WaymoValidation, LABEL_ID2NAME
import tqdm

In [ ]:
# Args are here: https://github.com/open-mmlab/mmdetection3d/blob/main/mmdet3d/datasets/waymo_dataset.py
waymo_base_kwargs = {
    "patch_height": 256,
    "patch_aspect_ratio": 1.3,
    "label_names": ['Car'],
    "data_root": "data/waymo",
    "pipeline": [],
    "box_type_3d": "Camera",
    "load_type": 'frame_based',
    "modality": dict(use_camera=True,use_lidar=False),
    "filter_empty_gt": False,
    "test_mode": False,
    "with_velocity": False,
    "use_valid_flag": False,
}

waymo_val = WaymoValidation(**waymo_base_kwargs)

In [ ]:
nusc_val[5]

In [ ]:
# mark patch_center_2d as red on full image
idx = 50
import matplotlib.pyplot as plt

for idx in tqdm.tqdm(range(len(nusc_val))):
    full_img = nusc_val[idx].full_img.permute(1,2,0)

    # mark patch_center_2d as red on full image 
    patch_center_2d = nusc_val[idx].patch_center_2d # torch.Size([2])

In [ ]:
patch = nusc_val[idx].patch.permute(1,2,0).numpy()
plt.imshow(patch)
plt.show(), idx

In [ ]:
# mark patch_center_2d as red on full image
idx = 11
import matplotlib.pyplot as plt

full_img = nusc_val[idx].full_img.permute(1,2,0)

# mark patch_center_2d as red on full image 
patch_center_2d = nusc_val[idx].patch_center_2d # torch.Size([2])

full_img = full_img.numpy()
full_img = full_img.copy() # (900, 1600, 3)

# make patch_center_2d as red with surrounding pixels red

patch_center_2d = patch_center_2d.numpy()
patch_center_2d = patch_center_2d.astype(int)
patch_center_2d = patch_center_2d.tolist()

for i in range(-5, 6):
    for j in range(-5, 6):
        full_img[patch_center_2d[1]+i, patch_center_2d[0]+j, :] = [1,0,0]
        
plt.imshow(full_img)
plt.show(), nusc_val[idx].pose_6d

In [ ]:
# visualize 

In [ ]:
nusc_val[0].full_img.shape, nusc_val[0].patch.shape

In [ ]:
nusc_val[0].full_img.min(), nusc_val[0].full_img.max()

In [ ]:
import matplotlib.pyplot as plt
nusc_val[0].full_img.shape # torch.Size([3, 900, 1600])
plt.imshow(nusc_val[0].full_img.permute(1,2,0))

In [ ]:
nusc_val[0].patch.min(), nusc_val[0].patch.max()

In [ ]:
plt.imshow(nusc_val[1].patch.permute(1,2,0))

In [ ]:
plt.imshow(nusc_val[0].patch.permute(1,2,0))

In [ ]:
nusc_val[1].patch.min(), nusc_val[1].patch.max()

In [ ]:
nusc_val[1].patch.max(), nusc_val[1].patch.min()

In [ ]:
#### The Below code may be based off the old test_nusc_dataloader.ipynb

In [ ]:
len(waymo_val)

In [ ]:
idx = 0
waymo_val[idx].cam_instances[0]

In [ ]:
num_car_instances = 0
num_instances = 0
pbar = tqdm.tqdm(total=len(waymo_val))
instance_counts = {k: 0 for k in LABEL_ID2NAME.keys()}
for sample_idx, sample in enumerate(waymo_val):
    
    for cam_instance in sample.cam_instances:
        instance_counts[cam_instance.bbox_label] += 1
        
        pbar.set_description(f"Car instances: {num_car_instances}, Total instances: {num_instances}, Current instance: {cam_instance.bbox_label}")
        num_instances += 1
        if cam_instance.bbox_label == 0:
            num_car_instances += 1
    pbar.update(1)
print(instance_counts)


In [ ]:
instance_counts

In [ ]:
class AverageMeter:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.history = []
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.history.append(val)
        
    def __str__(self):
        return f"{self.avg:.4f}"
    
    def __float__(self):
        return self.avg

In [ ]:
# compute  patch size statistics - mean mode median min max
patch_size_meter_x = AverageMeter()
patch_size_meter_y = AverageMeter()

pbar = tqdm.tqdm(waymo_val)
for sample in waymo_val:
    for cam_instance in sample.cam_instances:
        patch_size_meter_x.update(cam_instance.patch.shape[1])
        patch_size_meter_y.update(cam_instance.patch.shape[0])

        pbar.set_description(f"Patch size x: {patch_size_meter_x}, Patch size y: {patch_size_meter_y}")
        pbar.update(1)
        



In [ ]:
print(f"Mean patch size: {patch_size_meter_x}, {patch_size_meter_y}")
# print median, mode, min, max

import numpy as np
import scipy.stats as stats
print(f"Mean patch size x: {np.mean(patch_size_meter_x.history)}, y: {np.mean(patch_size_meter_y.history)}")
print(f"Median patch size x: {np.median(patch_size_meter_x.history)}, y: {np.median(patch_size_meter_y.history)}")
print(f"Mode patch size x: {stats.mode(patch_size_meter_x.history)}, y: {stats.mode(patch_size_meter_y.history)}")
print(f"Min patch size x: {np.min(patch_size_meter_x.history)}, y: {np.min(patch_size_meter_y.history)}")
print(f"Max patch size x: {np.max(patch_size_meter_x.history)}, y: {np.max(patch_size_meter_y.history)}")
print(f"Patch size x std: {np.std(patch_size_meter_x.history)}, y: {np.std(patch_size_meter_y.history)}")
print(f"Patch size x var: {np.var(patch_size_meter_x.history)}, y: {np.var(patch_size_meter_y.history)}")

In [ ]:
# histogram of patch sizes
import matplotlib.pyplot as plt
plt.hist(patch_size_meter_x.history, bins=100)
plt.show()
plt.hist(patch_size_meter_y.history, bins=100)
plt.show()


In [ ]:
# aspect ratio statistics
aspect_ratio_meter = AverageMeter()
pbar = tqdm.tqdm(waymo_val)
for sample in waymo_val:
    for cam_instance in sample.cam_instances:
        aspect_ratio_meter.update(cam_instance.patch.shape[1] / cam_instance.patch.shape[0])
        pbar.set_description(f"Aspect ratio: {aspect_ratio_meter}")
        pbar.update(1)
        
print(f"Mean aspect ratio: {aspect_ratio_meter}")
print(f"Median aspect ratio: {np.median(aspect_ratio_meter.history)}")
print(f"Mode aspect ratio: {stats.mode(aspect_ratio_meter.history)}")
print(f"Min aspect ratio: {np.min(aspect_ratio_meter.history)}")
print(f"Max aspect ratio: {np.max(aspect_ratio_meter.history)}")
print(f"Aspect ratio std: {np.std(aspect_ratio_meter.history)}")
print(f"Aspect ratio var: {np.var(aspect_ratio_meter.history)}")

# histogram of aspect ratios
import matplotlib.pyplot as plt
plt.hist(aspect_ratio_meter.history, bins=100)
plt.show()


In [ ]:
# histogram of patch sizes
import matplotlib.pyplot as plt
plt.hist(patch_size_meter_x.history, bins=100)
plt.show()
plt.hist(patch_size_meter_y.history, bins=100)

plt.show()

In [ ]:
# display 10 samples in the original size
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

pbar = tqdm.tqdm(waymo_val)
counter = 0
for i, sample in enumerate(pbar):
    for cam_instance_idx, cam_instance in enumerate(sample.cam_instances):
        pbar.set_description(f"Sample {i}, Camera {cam_instance_idx}, Label: {cam_instance.bbox_label}, Patch shape: {cam_instance.patch.shape}")
        if cam_instance.bbox_label == 0:
            plt.imshow(cam_instance.patch)
            plt.show()
            counter += 1
        if counter >= 10:
            break
        
